In [1]:
import os
import json
import csv
import requests


## Get from Trello

Go to this URL https://trello.com/b/vVQfKuJB.json in your browser and save under `inventory/`.

In [2]:

def archive_trello_json(trello_board, json_filename='vVQfKuJB.json'):
    ARCHIVES_BASE_DIR = 'inventory/trelloarchives'
    import datetime, os
    subdir = datetime.datetime.now().strftime('%Y-%m-%d__%Hh')
    archive_dir = os.path.join(ARCHIVES_BASE_DIR, subdir)
    if not os.path.exists(archive_dir):
        os.makedirs(archive_dir, exist_ok=True)
    json_path = os.path.join(archive_dir, json_filename)
    with open(json_path, 'w') as json_file:
        json.dump(trello_board, json_file, indent=4, sort_keys=True)
    
def get_trello_data(json_export_path='inventory/vVQfKuJB.json'):
    trello_board = json.load(open(json_export_path))
    
    archive_trello_json(trello_board)   # save a copy of the json for later diffing...

    # list columns
    status_columns = trello_board['lists']
    list_lookup = {}
    for status_column in status_columns:
        list_lookup[status_column['id']] = status_column['name']

    def should_reject(card):
        REJECT_PATTERNS = ['DEVELOPER', 'EVERYONE', 'IMPLEMENTATIONS',
                           'TECHNICAL LEAD', 'Implementations Members',
                            'TEMPLATE CARD', 'Technical Leads']
        if card['closed'] or '__' in card['name']:
            return True
        for p in REJECT_PATTERNS:
            if p in card['name']:
                return True
        return False

    # Extract only necessary info
    cards = trello_board['cards']
    trello_data = {}
    for card in cards:
        if should_reject(card):
            continue
        name = card['name']
        if name in trello_data:
            print('Found a duplicate card', name, card)
        
        trello_status = list_lookup[card['idList']]   # capital S to match notion json
        # Rename to Notion status
        if trello_status == 'Share With Provider':
            trello_status = 'Partner Review'
        if trello_status == 'Upcoming Sources':
            trello_status = 'Backlog'
        trello_data[name] = dict(
            id=card['id'],
            name=name,
            description=card['desc'],
            Status=trello_status
        )
    print('found', len(trello_data.keys()), 'Trello cards')

    return trello_data



trello_data = get_trello_data()

found 129 Trello cards


## Get notion.so data

1. Use the export function of the Cheffing Board
   https://www.notion.so/learningequality/515f1acd396641c39003020735d97500?v=955b63e44bed4263930176fee073e2fd
2. Unzip and copy the .csv file to `inverntory/` dir

In [3]:
DEFAULT_NOTION_CSV_PATH = 'inventory/Cheffing-Board-c70d925f-fd8f-4134-a26b-0b71a14bd6ff.csv'
def get_notion_data(notion_csv_path=DEFAULT_NOTION_CSV_PATH):
    reader = csv.DictReader(open(notion_csv_path))
    rows = list(reader)
    
    notion_data = {}
    for row in rows:
        row = dict(row)
        name = row['name']=row['\ufeffName']
        del row['\ufeffName']
        # name = row['name']
        notion_data[name] = row
    print('found', len(notion_data.keys()), 'Notion cards')
    
    return notion_data

notion_data = get_notion_data()

found 109 Notion cards


In [4]:
# Diff Trello and Notion
trello_names = set(trello_data.keys())
notion_names = set(notion_data.keys())

print("The following cards are not on Notion yet:")
for name in (trello_names - notion_names):
    print('  -', name)

notino_only_names = (notion_names - trello_names)
if notino_only_names:
    print("The following cards are on Notion but not on Trello:")
    for name in (notion_names - trello_names):
        print('  -', name)


The following cards are not on Notion yet:
  - MIT Blossoms
  - Sikana
  - Khan Academy (Burmese)
  - African Storybook
  - Khan Academy (Gujarati)
  - Developers (#sushi-chefs channel on slack)
  - Global Digital Library
  - Khan Academy
  - Engage NY (English)
  - Khan Academy (Korean)
  - Touchable Earth
  - CK-12
  - Tahrir Academy
  - GLEN world
  - Open Osmosis
  - Edraak
  - Lakou Kajou (Blue Butterfly Collaborative)
  - SDG Resource Center (RELX Group)
  - Be Internet Strong
  - OpenStax


In [5]:
# Status Diff
trello_names = set(trello_data.keys())
notion_names = set(notion_data.keys())
names = trello_names.intersection(notion_names)

for name in names:
    tc = trello_data[name]
    nc = notion_data[name]
    if tc['Status'] == nc['Status']:
        #print('OK:', name) 
        pass
    else:
        line = "{0:45}\t trello:{1:15}  notion:{2:12}".format(name[0:45], tc['Status'], nc['Status'])
        print(line)

Proyecto Descartes                           	 trello:Feedback needed  notion:In Development
Concord Consortium Science Simulations       	 trello:Ready for Chef   notion:Tech Specs Needed
Khan Academy (Armenian)                      	 trello:Partner Review   notion:Ready for Chef
Espresso English                             	 trello:QA Needed        notion:Feedback needed
Shamsuna Al-Arabiyah: MIT Basic Chemistry (Ar	 trello:Feedback needed  notion:QA Needed   
Free English with Hello Channel              	 trello:Feedback needed  notion:Tech Specs Needed
PraDigi (aka Pratham Open School)            	 trello:In Development   notion:Partner Review
Proyecto Biosfera                            	 trello:Feedback needed  notion:In Development
Al-Riyadiyat with Basil Al-Zubaidi العربيّة  	 trello:Feedback needed  notion:Ready for Chef
TED Talk and Video Lessons with (Multiple Lan	 trello:Feedback needed  notion:In Development
Hsoub Academy                                	 trello:QA Needed  